In [1]:
#########################
### get path of .keras  s folder as epochs of a training and visualize the metrics over them 


import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

folder_path = 'O:/project/epochs/sgd_fined_v2'
train_dir = 'O:/project/dataset/train' 
val_dir = 'O:/project/dataset/val'     
save_dir = folder_path + '/plots'  

os.makedirs(save_dir, exist_ok=True)

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(160, 160),  
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(160, 160), 
    batch_size=32,
    class_mode='categorical'
)

# Path to the folder containing the .keras files

epochs = []
train_accuracy = []
train_categorical_accuracy = []
train_precision = []
train_recall = []

val_accuracy = []
val_categorical_accuracy = []
val_precision = []
val_recall = []

for filename in sorted(os.listdir(folder_path)):
    if filename.endswith('.keras'):

        epoch_number = int(filename.split('_')[2].split('.')[0]) 
        epochs.append(epoch_number)
        

        model = tf.keras.models.load_model(os.path.join(folder_path, filename))
        
        train_metrics = model.evaluate(train_generator, verbose=0)
        val_metrics = model.evaluate(val_generator, verbose=0)
        
        train_accuracy.append(train_metrics[1])
        train_categorical_accuracy.append(train_metrics[2])
        train_precision.append(train_metrics[3])
        train_recall.append(train_metrics[4])

        val_accuracy.append(val_metrics[1])
        val_categorical_accuracy.append(val_metrics[2])
        val_precision.append(val_metrics[3])
        val_recall.append(val_metrics[4])

def create_save_and_pickle_plot(metric_name, train_values, val_values, epochs, save_dir):
    fig = plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_values, label=f'Training {metric_name}')
    plt.plot(epochs, val_values, label=f'Validation {metric_name}')
    plt.xlabel('Epochs')
    plt.ylabel(metric_name)
    plt.title(f'{metric_name} Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    save_path_png = os.path.join(save_dir, f'{metric_name}_over_epochs.png')
    fig.savefig(save_path_png)
    
    save_path_pickle = os.path.join(save_dir, f'{metric_name}_over_epochs.pkl')
    with open(save_path_pickle, 'wb') as f:
        pickle.dump(fig, f)
    
    plt.close(fig) 

    return fig

accuracy_fig = create_save_and_pickle_plot('Accuracy', train_accuracy, val_accuracy, epochs, save_dir)
categorical_accuracy_fig = create_save_and_pickle_plot('Categorical Accuracy', train_categorical_accuracy, val_categorical_accuracy, epochs, save_dir)
precision_fig = create_save_and_pickle_plot('Precision', train_precision, val_precision, epochs, save_dir)
recall_fig = create_save_and_pickle_plot('Recall', train_recall, val_recall, epochs, save_dir)


Found 5690 images belonging to 24 classes.
Found 1208 images belonging to 24 classes.


i:\anaconda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
